### Loading data from clouderizer drive

In [35]:
!ln -s ../data/ ./
!ln -s ../out/ ./

ln: failed to create symbolic link './data': File exists
ln: failed to create symbolic link './out': File exists


### Importing useful library

In [1]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing import image

Using TensorFlow backend.


In [5]:
# Convolutional Neural Network


# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 12, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('data/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('data/validation',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)



Found 4674 images belonging to 12 classes.
Found 759 images belonging to 12 classes.
Epoch 1/25
8000/8000 [==============================] - 706s 88ms/step - loss: 0.7059 - acc: 0.7660 - val_loss: 2.7924 - val_acc: 0.5523
Epoch 2/25
8000/8000 [==============================] - 737s 92ms/step - loss: 0.1597 - acc: 0.9480 - val_loss: 3.3653 - val_acc: 0.5560
Epoch 3/25
8000/8000 [==============================] - 743s 93ms/step - loss: 0.1016 - acc: 0.9677 - val_loss: 4.0814 - val_acc: 0.5322
Epoch 4/25
8000/8000 [==============================] - 741s 93ms/step - loss: 0.0799 - acc: 0.9743 - val_loss: 4.2046 - val_acc: 0.5191
Epoch 5/25
8000/8000 [==============================] - 745s 93ms/step - loss: 0.0692 - acc: 0.9783 - val_loss: 4.0981 - val_acc: 0.5586
Epoch 6/25
8000/8000 [==============================] - 760s 95ms/step - loss: 0.0629 - acc: 0.9799 - val_loss: 4.2283 - val_acc: 0.5692
Epoch 7/25
8000/8000 [==============================] - 769s 96ms/step - loss: 0.0553 - acc: 

### Checking the indices for each class 

In [6]:
training_set.class_indices

{'Ant-Man': 0,
 'Aquaman': 1,
 'Avengers': 2,
 'Batman': 3,
 'Black Panther': 4,
 'Captain America': 5,
 'Catwoman': 6,
 'Ghost Rider': 7,
 'Hulk': 8,
 'Iron Man': 9,
 'Spiderman': 10,
 'Superman': 11}

### Checking the files in test folder 

In [39]:
import os

path, dirs, files = next(os.walk("data/test_images/CAX_Superhero_Test"))
file_count = len(files)
file_count

3376

### Creating dataframe to store output for test folder

In [2]:
df= pd.DataFrame(columns=['filename','Superhero'])

### prediction

In [42]:
i=0
for img in os.listdir('data/test_images/CAX_Superhero_Test'):
    print(img)
    if img != '.DS_Store':
        test_image = image.load_img('data/test_images/CAX_Superhero_Test/'+str(img), target_size = (64, 64))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = classifier.predict(test_image)
        print(result)
        for ind,x in enumerate(np.nditer(result)):
            if x==1.0:
                prediction = list(training_set.class_indices.keys())[list(training_set.class_indices.values()).index(ind)]
        df.loc[i] = [img,prediction]
        i+=1
        print(prediction)

print (i)

superhero_test_1310.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_3072.jpg
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ant-Man
superhero_test_1434.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_95.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2994.jpg
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
Black Panther
superhero_test_1688.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_179.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_217.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_458.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_1373.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_377.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_3360.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_867.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_241.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_2381.jpg
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Captain America
superhero_test_1774.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_3352.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1001.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_588.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
Hulk
superhero_test_3336.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_2564.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_2173.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1458.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_1365.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_32.jpg
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ant-Man
superhero_test_2662.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_25.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
Hulk
superher

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2839.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_2114.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_2170.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_2059.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2725.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2434.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2459.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2336.jpg
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ant-Man
superhero_test_321.jpg
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Avengers
superhero_test_1045.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_3031.jpg
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Avengers
superhero_test_2871.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2201.jpg
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Avengers
superhero

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
Hulk
superhero_test_559.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_2407.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1946.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_3292.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_147.jpg
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Avengers
superhero_test_178.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_319.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1796.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_1787.jpg
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0679199e-21
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00]]
Iron Man
superhero_test_181.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_564.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_460.jpg
[[0. 0

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_1126.jpg
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Captain America
superhero_test_2298.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2330.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2219.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_1259.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1531.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_1877.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2975.jpg
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 2.4230041e-18
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
Batman
superhero_test_3236.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_507.jpg
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
Black Panther
superhero_test_2453.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_t

...

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_1752.jpg
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ant-Man
superhero_test_245.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_3039.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_2090.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_2032.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_325.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_1011.jpg
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ant-Man
superhero_test_227.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2770.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_3008.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2890.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_876.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_541.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_2

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_3139.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2487.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_2650.jpg
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ant-Man
superhero_test_865.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_1148.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_573.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_2880.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_1926.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2647.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2480.jpg
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Captain America
superhero_test_672.jpg
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Catwoman
superhero_test_2554.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1737.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
super

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Captain America
superhero_test_3363.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_3075.jpg
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Avengers
superhero_test_2776.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_1306.jpg
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
Black Panther
superhero_test_3196.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
Hulk
superhero_test_472.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2777.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_413.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1293.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_452.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2588.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_2302.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_3069.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
supe

Batman
superhero_test_856.jpg
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_2812.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Superman
superhero_test_1401.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_1135.jpg
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Catwoman
superhero_test_2069.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_1707.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2497.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Spiderman
superhero_test_2174.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
Hulk
superhero_test_1176.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_270.jpg
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Catwoman
superhero_test_1782.jpg
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Batman
superhero_test_2891.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_647.jpg
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Captain America
superhero_test_3332.jpg
[[0.0000000e+00 0.0000000e

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Aquaman
superhero_test_3245.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_825.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
superhero_test_2492.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
Iron Man
3375


In [43]:
df

,filename,Superhero
0,superhero_test_1310.jpg,Iron Man
1,superhero_test_3072.jpg,Ant-Man
2,superhero_test_1434.jpg,Batman
3,superhero_test_95.jpg,Batman
4,superhero_test_2994.jpg,Black Panther
5,superhero_test_1688.jpg,Aquaman
6,superhero_test_179.jpg,Spiderman
7,superhero_test_217.jpg,Iron Man
8,superhero_test_458.jpg,Superman
9,superhero_test_1373.jpg,Spiderman


### saving into a csv file 

In [44]:
df.to_csv('Superhero_3375_SampleSubmission.csv',sep=',',index=False)